学生のパフォーマンス

データセットの説明：

このデータは、ポルトガルの2008個の学校の中等教育における生徒の成績から取得したデータである。今回は数学にフォーカスしたデータを扱う。


特徴量：

sex:性別(F or M)

age:15~22の年齢

famsize:家族の規模(LE3:3人以下、GT3:３人以上)

Pstatus:親の同棲状況(T:同居、A:離れて)

Medu:母親の教育(0:なし、1:初等教育、2:5-9年生、3:中等教育、4:高等教育)

Fedu:父親の教育(0:なし、1:初等教育、2:5-9年生、3:中等教育、4:高等教育)

traveltime:自宅から学校までの移動時間(数値:1〜<15分、2〜15〜30分、3〜30分〜1時間、または4〜>1時間)

studytime:毎週の学習時間(数値:1 - <2時間、2 - 2 - 5時間、3 - 5から10時間、または4 - >10時間)

nursery:保育園に通った(バイナリ:はいまたはいいえ)

internet：-自宅でのインターネットアクセス(バイナリ:はいまたはいいえ)

romantic:ロマンチック-ロマンチックな関係(バイナリ: はいまたはいいえ)

famrel:家族関係の質 (数値: 1 - 非常に悪いから 5 - 非常に良い)

health:現在の健康状態 (数値: 1 - 非常に悪いから 5 - 非常に良い)

absences:欠席数 (数値: 0 から 93)

G3:最終成績

必要なモジュールのインポート

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from sklearn.metrics import mean_squared_error, r2_score

データの取得

In [7]:
#今回利用するデータは区切り文字が";"であるためsepに設定する
df = pd.read_csv('student-mat.csv',sep=";")

print(df)
print(df.shape)
print(df.columns)

    school sex  age address famsize Pstatus  ...  Walc  health absences  G1  G2  G3
0       GP   F   18       U     GT3       A  ...     1       3        6   5   6   6
1       GP   F   17       U     GT3       T  ...     1       3        4   5   5   6
2       GP   F   15       U     LE3       T  ...     3       3       10   7   8  10
3       GP   F   15       U     GT3       T  ...     1       5        2  15  14  15
4       GP   F   16       U     GT3       T  ...     2       5        4   6  10  10
..     ...  ..  ...     ...     ...     ...  ...   ...     ...      ...  ..  ..  ..
390     MS   M   20       U     LE3       A  ...     5       4       11   9   9   9
391     MS   M   17       U     LE3       T  ...     4       2        3  14  16  16
392     MS   M   21       R     GT3       T  ...     3       3        3  10   8   7
393     MS   M   18       R     LE3       T  ...     4       5        0  11  12  10
394     MS   M   19       U     LE3       T  ...     3       5        5   8 

データの整形

In [8]:
#入力(特徴量)と出力(目的変数)の分割
X = df.loc[:,['sex','age','famsize','Pstatus','Medu','Fedu','traveltime','studytime','nursery','internet','romantic','famrel','health','absences']]
le = LabelEncoder()
#ラベルエンコーディング

X['sex'] = le.fit_transform(df['sex'])
X['famsize'] = le.fit_transform(df['famsize'])
X['Pstatus'] = le.fit_transform(df['Pstatus'])
X['nursery'] = le.fit_transform(df['nursery'])
X['internet'] = le.fit_transform(df['internet'])
X['romantic'] = le.fit_transform(df['romantic'])

X = (X - X.mean())/X.std()

y = df['G3']

print(y)
#converted_y = le.fit_transform(y)
#print(y)
#print(converted_y)

0       6
1       6
2      10
3      15
4      10
       ..
390     9
391    16
392     7
393    10
394     9
Name: G3, Length: 395, dtype: int64


データの分割

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(276, 14) (119, 14) (276,) (119,)


モデルの定義

In [17]:
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

モデルの学習

In [18]:
model.fit(X_train, y_train, epochs=1000, batch_size=32, verbose=0)

モデルの予測

In [19]:
y_pred = model.predict(X_test)
y_pred = np.reshape(y_pred, y_test.shape)

4/4 [==============================] - 0s 2ms/step


回帰モデルの評価

In [20]:
# 評価（平均二乗誤差）
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

Mean Squared Error: 30.506186506895084


In [21]:
# 評価（R2スコア）
r2 = r2_score(y_test, y_pred)
print("R2 Score:", r2)

R2 Score: -0.38777629437736527


モデルの保存

In [15]:
model.save('academy_mlp.h5')